In [8]:
import pandas as pd
import json
import glob
import os

## I prompted and iterated this code with Gemini.

In [6]:
def collate_json_files(directory, pattern="Streaming_History_Audio_*.json"):
    """
    Collates multiple JSON files in a directory into a single Pandas DataFrame.

    Args:
        directory (str): The directory containing the JSON files.
        pattern (str, optional): The glob pattern to match the JSON files. Defaults to "Streaming_History_Audio_*.json".

    Returns:
        pandas.DataFrame: A DataFrame containing the combined data, or None if an error occurs.
    """

    json_files = glob.glob(os.path.join(directory, pattern))
    all_data = []

    for file in json_files:
        try:
            with open(file, 'r', encoding='utf-8') as f:
                data = json.load(f)

                if isinstance(data, list) and all(isinstance(item, dict) for item in data):
                    all_data.extend(data)
                elif isinstance(data, dict):
                    all_data.append(data)
                else:
                    print(f"Warning: JSON structure in {file} is not a list of dictionaries or a dictionary. Skipping.")

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in {file}: {e}")
        except FileNotFoundError:
            print(f"Error: File not found at {file}")
        except Exception as e:
            print(f"An unexpected error occurred processing {file}: {e}")

    if all_data:
        df = pd.DataFrame(all_data)
        return df
    else:
        return None


In [9]:
# --- Main script ---
directory_path = "../Data/my_spotify_data/Spotify Extended Streaming History/"  # Replace with the actual path

# Collate Audio Data
df_audio = collate_json_files(directory_path, "Streaming_History_Audio_*.json")

if df_audio is not None:
    print(f"Successfully loaded {len(df_audio)} audio records.")

    # Collate Video Data (separately)
    df_video = collate_json_files(directory_path, "Streaming_History_Video_*.json")

    if df_video is not None:
        print(f"Successfully loaded {len(df_video)} video records.")

        # Save to Parquet (separate files)
        df_audio.to_parquet(os.path.join(directory_path, "collated_spotify_audio.parquet"))
        df_video.to_parquet(os.path.join(directory_path, "collated_spotify_video.parquet"))

        print("Data saved to Parquet files.")

    else:
        print("No video data was loaded. Check the file or path.")

else:
    print("No audio data was loaded. Check the files or path.")

Successfully loaded 129958 audio records.
Successfully loaded 147 video records.
Data saved to Parquet files.
